<a href="https://colab.research.google.com/github/gabrielkerr/deep_learning_fall_2018/blob/master/Keras_Content_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
# Installing packages and cloning git repos
!pip install music21
!pip install h5py
!git clone https://github.com/gabrielkerr/Classical-Piano-Composer.git

fatal: destination path 'Classical-Piano-Composer' already exists and is not an empty directory.


In [0]:
# switch directories to the cloned repo with defined helper functions
os.chdir('Classical-Piano-Composer')

In [2]:
# imports
import keras
from keras import backend as K
import tensorflow as tf
import music21
import h5py
import os
from lstm import get_notes, prepare_sequences
import numpy as np

Using TensorFlow backend.


# Get Features

In [5]:
# Get notes from Beethoven files
notes = get_notes()

Parsing beeth/appass_3.mid
Parsing beeth/mond_3.mid
Parsing beeth/beethoven_hammerklavier_4.mid
Parsing beeth/elise.mid
Parsing beeth/beethoven_les_adieux_3.mid
Parsing beeth/waldstein_1.mid
Parsing beeth/beethoven_opus10_2.mid
Parsing beeth/beethoven_opus22_2.mid
Parsing beeth/beethoven_hammerklavier_1.mid
Parsing beeth/beethoven_les_adieux_2.mid
Parsing beeth/beethoven_opus90_1.mid
Parsing beeth/waldstein_2.mid
Parsing beeth/pathetique_1.mid
Parsing beeth/beethoven_hammerklavier_2.mid
Parsing beeth/pathetique_2.mid
Parsing beeth/appass_1.mid
Parsing beeth/beethoven_opus22_4.mid
Parsing beeth/beethoven_opus22_3.mid
Parsing beeth/appass_2.mid
Parsing beeth/waldstein_3.mid
Parsing beeth/mond_1.mid
Parsing beeth/beethoven_opus22_1.mid
Parsing beeth/beethoven_hammerklavier_3.mid
Parsing beeth/beethoven_opus90_2.mid
Parsing beeth/beethoven_opus10_1.mid
Parsing beeth/beethoven_opus10_3.mid
Parsing beeth/pathetique_3.mid
Parsing beeth/beethoven_les_adieux_1.mid
Parsing beeth/mond_2.mid


In [0]:
# Get count of unique "vocabulary" (notes)
n_vocab = len(set(notes))

In [0]:
# Extract sequences of 100 notes from midis
network_input, network_output = prepare_sequences(notes, n_vocab)

# Naive Loss Function

Content loss is defined as the norm between `input_sequence` - `output_sequence`.

Here we perform content loss between the 1st and 2nd input sequences to show functionality of the loss function, but ideally a sequence of 100 notes would be generated from the trained network and then compared with the input sequence.

In [0]:
def content_loss(input_sequence, output_sequence):
  return np.linalg.norm(input_sequence - output_sequence)

In [9]:
content_loss(network_input[0], network_input[1])

2.348897917869448

In [0]:
from keras.layers import LSTM, Dropout, Dense, Activation
from keras import Sequential

model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(359))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights.hdf5')

In [11]:
len(model.get_weights()[0])

359

In [18]:
import tensorflow as tf
with tf.Session() as sess:
  init = tf.global_variables_initializer()
  sess.run(init)
  v = sess.run(model.weights[0])
  print(v) 
  assign_op = model.weights[0].assign(np.zeros(v.shape))
  sess.run(assign_op)
  print(model.weights[0].eval())

[[ 0.01884716 -0.035212   -0.04476875 ... -0.03120076 -0.00696015
  -0.01515903]]
[[0. 0. 0. ... 0. 0. 0.]]
